## True Random dice!

This is a notebook to make true random dice out of qubits.
Eventually, this should be able to parse various input strings:
e.g. "3d4" is calculated by creating a 4-state (2 qubit) circuit and running it three times.
For dice sizes that aren't created easily with h gates, make the smallest quantum circuit needed and reroll anything too high.

In [154]:
#required input just for rolls
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, transpile
from qiskit_aer import AerSimulator
import numpy as np

In [226]:
def handle_input(strg):
    #this func will take an input of the form 3d6
    #then split to calls = 3, size = 6
    split = strg.split("d")
    return(int(split[0]),int(split[1]))


def create_circuit(size):
    #take size as int, build a circuit with equal chances of each state
    qubits = int(np.ceil(np.log2(size)))
    qr = QuantumRegister(qubits,'x')
    cr = ClassicalRegister(qubits,'c')
    qc = QuantumCircuit(qr,cr)       
    for i in range (qubits):
        qc.h(i)        
    qc.measure(qr,cr)
    return qc


def count_to_dec(count):
    #takes result.get_counts() and converts to a decimal result
    strg = str(count)
    split = strg.split("'")
    num = int(split[1],2)
    return (num+1)
    
    
    
def main(inp):
    #take in the roll text and run everything!
    calls,size = handle_input(inp)
    qc = create_circuit(size)
    simulator = AerSimulator()
    compiled_circuit = transpile(qc, simulator)
    total_roll = 0
    
    for _ in range (calls):
        reroll = True
        while reroll:
            #sym and run the circuit
            job = simulator.run(compiled_circuit, shots=1)
            result = job.result()
            roll = count_to_dec(result.get_counts())
            #reroll if result is too big
            if roll <= size:
                reroll = False
        total_roll+=roll
        
    print ("you rolled",inp,"and got a",total_roll,"!")


In [231]:
#main
main('1d20')

you rolled 1d20 and got a 8 !
